# Analyzing Sea Level Rise in the Cloud with Coiled and Earthaccess

_This notebook was adapted from [this NASA Earthdata Cloud Cookbook example](https://nasa-openscapes.github.io/earthdata-cloud-cookbook/tutorials/Sea_Level_Rise.html)_

## Get data files with `earthaccess`

In [ ]:
%%time

import earthaccess

# Retrieve data files for the dataset I'm interested in
granules = earthaccess.search_data(
    short_name="SEA_SURFACE_HEIGHT_ALT_GRIDS_L4_2SATS_5DAY_6THDEG_V_JPL2205",
    temporal=("2000", "2019"),
)

## Define processing function

In [ ]:
import os
import coiled
import fsspec
import xarray as xr

@coiled.function(region="us-west-2")  # Same region as data
def process(granule, fs):
    files = []
    for file in granule.data_links("direct"):
        ds = xr.open_dataset(fs.open(file))
        ds = ds.sel(Latitude=slice(23, 50), Longitude=slice(270, 330))
        ds = ds.SLA.where((ds.SLA >= 0) & (ds.SLA < 10))
        # Write processed data to S3
        outfile = f"s3://openscapes-scratch/agu-2023/{os.path.basename(file)}"
        with fsspec.open(f"simplecache::{outfile}", mode="wb") as out:
            ds.to_netcdf(out, engine="h5netcdf")
        files.append(outfile)
    return files


@coiled.function(region="us-west-2")  # Same region as data
def plot(files):
    fs = fsspec.filesystem("s3")
    ds = xr.open_mfdataset(
        [fs.open(f, mode="rb") for f in files],
        concat_dim="Time",
        combine="nested",
        parallel=True,
    )
    return ds.SLA.std("Time").plot(x="Longitude", y="Latitude", figsize=(14, 6))

## Process Granules

In [ ]:
fs = earthaccess.get_s3fs_session(results=granules)

### Single file

In [ ]:
%%time

files = process(granules[0], fs=fs)
plot(files);

### All files in parallel

In [ ]:
%%time

files = process.map(granules, fs=fs)
files = sum(files, [])  # Flatten list of files
plot(files);